In [1]:
import os

In [2]:
%pwd

'd:\\ML_Projects\\Resume Project\\Project_6_insurance_claim_prediction\\reasearch'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML_Projects\\Resume Project\\Project_6_insurance_claim_prediction'

In [36]:
# ENtity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    # mlflow_uri: str

In [37]:
# prepare confuguration manager class
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml,create_directories

In [29]:
%pip install -q dagshub mlflow


^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import dagshub
dagshub.init(repo_owner='rajuu9825', repo_name='my-first-repo', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\ML_Projects\Resume Project\Project_6_insurance_claim_prediction\venv\Lib\site-packages\rich\live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=e51fbe5b-f457-4611-8f8e-bda31cab3ae7&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=89f27841f8ed86b4b576a875e42ce9c868e7ec32f34186dafd2f8e134fde0246


[2025-04-10 17:34:57,180: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2025-04-10 17:34:58,100: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2025-04-10 17:34:58,901: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as rajuu9825

[2025-04-10 17:34:58,908: INFO: helpers: Accessing as rajuu9825]
[2025-04-10 17:35:00,209: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/rajuu9825/my-first-repo "HTTP/1.1 200 OK"]
[2025-04-10 17:35:01,093: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "rajuu9825/my-first-repo"

[2025-04-10 17:35:01,101: INFO: helpers: Initialized MLflow to track repo "rajuu9825/my-first-repo"]


Repository rajuu9825/my-first-repo initialized!

[2025-04-10 17:35:01,106: INFO: helpers: Repository rajuu9825/my-first-repo initialized!]


In [38]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
            config =  self.config.model_evaluation
            params = self.params.ElasticNet
            schema = self.schema.TARGET_COLUMN

            create_directories([config.root_dir])

            model_trainer_config = ModelEvaluationConfig(
                root_dir = config.root_dir,
                test_data_path =  config.test_data_path,
                model_path = config.model_path,
                all_params = params,
                metric_file_name=config.metric_file_name,
                target_column=schema.name,
                # mlflow_uri="https://dagshub.com/rajuu9825/End-to-end-Machine-Learning-Project-with-MLFlow.mlflow"
            )

            return model_trainer_config

In [39]:
import os
import pandas as pd
from src.mlproject.utils.common import save_json
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlflow.models.signature import infer_signature

In [40]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        
        return rmse,mae,r2
    
    def log_into_mlflow(self):

        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Set MLflow Tracking URI (DagsHub or local)
        # mlflow.set_tracking_uri(self.config.mlflow_uri)  # e.g., https://dagshub.com/your_username/your_repo.mlflow

        # Optional: set username/token as env vars if needed
        # os.environ["MLFLOW_TRACKING_USERNAME"] = "your_username"
        # os.environ["MLFLOW_TRACKING_PASSWORD"] = "your_token"

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Set or create experiment
        mlflow.set_experiment("PolicyPredict")

        with mlflow.start_run():

            # Predictions and signature
            predicted_qualities = model.predict(test_x)
            signature = infer_signature(test_x, predicted_qualities)

            # Evaluation
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally (optional)
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log params and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Log model and register it
            # if tracking_url_type_store != "file":
            #     mlflow.sklearn.log_model(
            #         sk_model=model,
            #         artifact_path="model",
            #         registered_model_name="PolicyPredict",  # This name will show in Model Registry
            #         signature=signature
            #     )
            # else:
            #     mlflow.sklearn.log_model(
            #         sk_model=model,
            #         artifact_path="model",
            #         signature=signature
            #     )

            print("MLflow logging complete 🚀")


In [41]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-04-10 18:01:33,886: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-10 18:01:33,888: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-10 18:01:33,891: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-10 18:01:33,894: INFO: common: Created directory at: artifacts]
[2025-04-10 18:01:33,896: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-04-10 18:01:35,086: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
MLflow logging complete 🚀
🏃 View run burly-kit-325 at: https://dagshub.com/rajuu9825/my-first-repo.mlflow/#/experiments/1/runs/a1a0805184a941268c94c31e969c5054
🧪 View experiment at: https://dagshub.com/rajuu9825/my-first-repo.mlflow/#/experiments/1
